In [ ]:
import ctf
import numpy as np
import numpy.linalg as la
from ctf import random
glob_comm = ctf.comm()

In [ ]:
def normalize(Z):
    norms = ctf.tensor(r)
    norms.i("u") << Z.i("pu")*Z.i("pu")
    norms = 1./norms**.5
    X = ctf.tensor(copy=Z)
    Z.set_zero()
    Z.i("pu") << X.i("pu")*norms.i("u")
    return 1./norms

In [ ]:
def updateOmega(I,J,K,sparsity):
    '''
    Gets a random subset of rows for each U,V,W iteration
    '''
    Actf = ctf.tensor((I,J,K),sp=True)
    Actf.fill_sp_random(0,1,sparsity)
    omegactf = ((Actf > 0)*ctf.astensor(1.))
    return omegactf


def getIndexFromOmega(omega,I,J,K):
    '''
    Gets a random i,j,k contained in Ω[0...i-1,0...j-1,0...k-1]
    '''
    #TODO: Better sampling instead of MC.
    i = random.randint(0,I-1)
    j = random.randint(0,J-1)
    k = random.randint(0,K-1)
    while omega[i][j][k] != 1:
        i = random.randint(0,I-1)
        j = random.randint(0,J-1)
        k = random.randint(0,K-1)
        
    return (i,j,k)

In [ ]:
def updateU(T,V,W,regParam,omega,I,J,K,r):
    '''Update U matrix by using the formula'''
    
    M = ctf.tensor((J,K,r))
    M.i("jku") << V.i("ju")*W.i("ku")
    [U_,S_,V_] = ctf.svd(M.reshape((J*K,r)))
    S_ = 1./S_
    U.set_zero()
    U.i("iu") << V_.i("vu")*S_.i("v")*U_.reshape((J,K,r)).i("jkv")*T.i("ijk")
    normalize(U)
    
    return U   
    
    
def updateV(T,U,W,regParam,omega,I,J,K,r):
    '''Update V matrix by using the formula'''
    
    M = ctf.tensor((I,K,r))
    M.i("iku") << U.i("iu")*W.i("ku")
    [U_,S_,V_] = ctf.svd(M.reshape((I*K,r)))
    S_ = 1./S_
    V.set_zero()
    V.i("ju") << V_.i("vu")*S_.i("v")*U_.reshape((I,K,r)).i("ikv")*T.i("ijk")
    normalize(V)
    
    return V   

def updateW(T,U,V,regParam,omega,I,J,K,r):
    '''Update V matrix by using the formula'''
    
    M = ctf.tensor((I,J,r))
    M.i("iju") << U.i("iu")*V.i("ju")
    [U_,S_,V_] = ctf.svd(M3.reshape((I*J,r)))
    S_ = 1./S_
    W.set_zero()
    W.i("ku") << V_.i("vu")*S_.i("v")*U_.reshape((I,J,r)).i("ijv")*T.i("ijk")
    normalize(W)
    
    return W

In [ ]:
def getALSCtf(T,U,V,W,regParam,omega,I,J,K,r):
    '''
    Same thing as above, but CTF
    '''
    E = ctf.tensor((I,J,K))
    E.i("ijk") << T.i("ijk") - omega("ijk")*U.i("iu")*V.i("ju")*W.i("ku")
    curr_err_norm = ctf.vecnorm(E) + (ctf.vecnorm(U) + ctf.vecnorm(V) + ctf.vecnorm(W))*regParam
    
    while True:
        U = updateU(T,V,W,regParam,omega,I,J,K,r) 
        V = updateV(T,U,W,regParam,omega,I,J,K,r) 
        W = updateW(T,U,V,regParam,omega,I,J,K,r) 
        E.i("ijk") << T.i("ijk") - omega("ijk")*U.i("iu")*V.i("ju")*W.i("ku")
        next_err_norm = ctf.vecnorm(E) + (ctf.vecnorm(U) + ctf.vecnorm(V) + ctf.vecnorm(W))*regParam
        
        if curr_err_norm - next_err_norm > .001:
            break
            
        print(curr_err_norm, next_err_norm)
        curr_err_norm = next_err_norm
        
    return U,V,W

In [ ]:
I = 3
J = 4
K = 5
r = 3 # TODO: determine rank?
sparsity = .2
learningRate = .1
regParamALS = 2
regParamSGD = .1
width = 1  # how many random i,j we're going to get
convNo = .00002  # When we'll stop converging
  
# 3rd-order tensor
A = ctf.tensor((I,J,K),sp=True)
A.fill_sp_random(0,1,1)

ctf.random.seed(42)
U = ctf.random.random((I,r))
V= ctf.random.random((J,r))
W= ctf.random.random((K,r))

normalize(U)
normalize(V)
normalize(W)

omega = updateOmega(I,J,K,sparsity)